## Inicializacao

In [1]:
import numpy as np
import pandas as pd
import sympy as sp
import scipy.io as sc 
from sympy import symbols, pprint
import matplotlib.pyplot as plt
from structureSelector import *
from methods.utils.utilities import *

ballbeam  = np.loadtxt('data/ballbeam.dat')
print(ballbeam.shape)
part = ballbeam.shape[0] // 2#700

u = ballbeam[:part, 0].reshape((1,-1))
y = ballbeam[:part, 1].reshape((1,-1))

'''df = pd.DataFrame(y.T, columns=['Column_A'])
p = df.rolling(5).mean()
p[:2] = 0
y = np.array(p['Column_A']).reshape((1,-1))
y[:, 1:] = (y[:, 1:] - y[:, :-1]) / 0.1
print(u.shape, y.shape)'''

(1000, 2)


"df = pd.DataFrame(y.T, columns=['Column_A'])\np = df.rolling(5).mean()\np[:2] = 0\ny = np.array(p['Column_A']).reshape((1,-1))\ny[:, 1:] = (y[:, 1:] - y[:, :-1]) / 0.1\nprint(u.shape, y.shape)"


## Selecao de estrutura

In [ ]:
#Selecione o tanque 
output = 0  # 0 ou 1

num = [5, 5]
params = []
params.append({'nb':[3],'na':[3], 'level':1, 'nonlinear':[1,1,0,1,0], 'root':False, 'delay':1, 'diff':True})
#params.append({'nb':[0,2],'na':[1], 'level':1, 'nonlinear':[0,0,0,0,0], 'root':True, 'delay':0})

sselector = structureSelector()
ss = sselector.symbolic_regressors(**params[output])
#print(ss)

vCandidatos = sselector.matrix_candidate(u, y, **params[output], dt=0.1)

pad = max(max(params[output]['nb']), max(params[output]['na']))
psi, selected  = sselector.semp(vCandidatos.T, y[output, pad:], num[output], 0.000000001)
model = ss[selected]
#print(model, selected)
theta = LSM(y[output, pad:], psi)

print(model, theta)

slivre = sselector.predict(u, y, theta, ss[selected], params[output]['nb'], params[output]['na'], output, params[output]['delay'], diff=True, dt=0.1)
yhat = sselector.oneStepForward(u, y, theta, ss[selected], params[output]['nb'], params[output]['na'], output, diff=True, dt=0.1)

def metrics(y, yest):
    residuo1 = y - yest
    mape = round(np.mean(np.abs(residuo1 / (yest + np.finfo(np.float64).eps))), 5)
    print('MSE:', np.mean(np.square(residuo1)), '\nAET:', np.sum(np.abs(residuo1)), '\nMAPE:', str(mape) + '%')
    cc = np.corrcoef(y, yest)
    #print("Correlation pearson:", np.mean(cc))

print("\nSimulação livre")
metrics(y, slivre)
print("\nUm passo a frente")
metrics(y, yhat)

In [ ]:
psi

### Simulação

In [ ]:
plt.figure(figsize=(15,4))
plt.title("Tanque " + str(output+1))
plt.plot(y[output].T, label='Original')
plt.plot(yhat, label='um passo a frente')
plt.plot(slivre, label='Livre')
plt.legend()
plt.show()

In [ ]:
df = pd.DataFrame(y.T, columns=['Column_A'])
p = df.rolling(2).mean()
p[:2] = 0
p = np.array(p['Column_A'])
v = (p[1:] - p[0:-1])
a = (v[1:] - v[:-1])/0.1
z = np.zeros(y.shape)

#plt.plot(y.T)
plt.plot(u.T)
plt.plot(p)
#plt.plot(v)
#plt.plot(a)
'''plt.plot(u.T)
plt.plot(z.T)'''
plt.legend()
plt.show()

In [ ]:
du = np.zeros(u.shape)
du[:, 1:] = (u[:, 1:] - u[:, :-1]) / 0.1

uu = np.zeros(u.shape)
yy = np.zeros(y.shape)
for i in range(1, u.shape[1]):
    uu[:, i] = uu[:, i-1] + u[:, i] * 0.1
    yy[:, i] = yy[:, i-1] + (y[:,i-1] * du[:,i-1]**2)
    
e = 1
pp = np.vstack(( uu[:, e:], yy[:, e:])).T

theta = LSM(y[:, e:], pp)
print(theta)

r = np.zeros(y.shape)
for i in range(1, u.shape[1]):
    r[:, i] = r[:, i] * theta[0] + theta[2] * (r[:,i-1] * du[:,i]**2)

In [ ]:
plt.plot(y.T)
plt.plot(uu.T)
plt.plot(yy.T)
plt.plot(r.T)
#plt.plot(uu.T + yy.T * 100)
#plt.plot(pp @ theta)
plt.show()

In [ ]:


plt.show()